# Experimenting Localization with SIFT Matching and Essential Matrix

This script takes 3 images from the training dataset, attempting to use the first 2 images to predict the other one's location

In [399]:
import numpy as np
import pandas as pd
import cv2

# Steps
* Set input image filenames to variables `i1`,`i2`
* Set validation image filename to variable `i3`
* Run onwards

In [674]:
i1 = 'IMG3200_1'
i2 = 'IMG3640_2'
i3 = 'IMG3001_1'

img = cv2.imread('./train/' + i1 + '.jpg')
img2 = cv2.imread('./train/' + i2 + '.jpg')
img3 = cv2.imread('./train/' + i3 + '.jpg')

# SIFT keypoints
sift = cv2.SIFT_create()
kp,des = sift.detectAndCompute(img,None)
kp2,des2 = sift.detectAndCompute(img2,None)
kp3,des3 = sift.detectAndCompute(img3,None)

In [675]:
# FLANN matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

# Limit for Lowe's Ratio test
ratio = 0.6
# 8 points are needed for 8-point algorithm
MIN_MATCH_NUM = 8

# Matching for each pair of image
matches = flann.knnMatch(des,des2,k=2)
good = [m for m,n in matches if m.distance < ratio*n.distance]
if len(good)> MIN_MATCH_NUM:
    pts12 = np.float32([kp[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    pts21 = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

matches = flann.knnMatch(des,des3,k=2)
good = [m for m,n in matches if m.distance < ratio*n.distance]
if len(good)> MIN_MATCH_NUM:
    pts13 = np.float32([kp[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    pts31 = np.float32([kp3[m.trainIdx].pt for m in good]).reshape(-1,1,2)

matches = flann.knnMatch(des2,des3,k=2)
good = [m for m,n in matches if m.distance < ratio*n.distance]
if len(good)> MIN_MATCH_NUM:
    pts23 = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    pts32 = np.float32([kp3[m.trainIdx].pt for m in good]).reshape(-1,1,2)

In [676]:
# Intrinsic Camera Matrix
FOV_X = 73.3*np.pi/180
FOV_Y = 53.1*np.pi/180

cx = img.shape[1]/2
cy = img.shape[0]/2

fx = cx/np.tan(FOV_X/2)
fy = cy/np.tan(FOV_Y/2)

K = np.array([[fy,0,cy],
              [0,fx,cx],
              [0,0,1]])

In [677]:
# Get the respective rotation and translation
E,_ = cv2.findEssentialMat(pts12,pts21,K,method=cv2.FM_LMEDS)
_,R,T,_ = cv2.recoverPose(E,pts12,pts21,K)

E1,_ = cv2.findEssentialMat(pts13,pts31,K,method=cv2.FM_LMEDS)
_,R1,T1,_ = cv2.recoverPose(E1,pts13,pts31,K)

E2,_ = cv2.findEssentialMat(pts23,pts32,K,method=cv2.FM_LMEDS)
_,R2,T2,_ = cv2.recoverPose(E2,pts23,pts32,K)

In [678]:
# Print out coordinates to know what to expect
train_xy = pd.read_csv('train.csv')
pt1 = train_xy.loc[train_xy['id']==i1,['x','y']].values
pt2 = train_xy.loc[train_xy['id']==i2,['x','y']].values
pt3 = train_xy.loc[train_xy['id']==i3,['x','y']].values
print(pt1,pt2,pt3)
print(pt1-pt2)

[[ 46.31932192 -53.81728035]] [[ 49.01932192 -57.01728035]] [[ 50.51932192 -56.81728035]]
[[-2.7  3.2]]


In [679]:
# Displacement Vector
dp = (pt1-pt2)

In [1]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    
    Params
    ---
    - vec1: A 3d "source" vector
    - vec2: A 3d "destination" vector
    
    Returns
    ---
    mat: A transform matrix which when applied to vec1, aligns it with vec2.
    """
    a = (vec1 / np.linalg.norm(vec1)).reshape(3)
    b = (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [683]:
# Confirm correct mapping, by ensuring that rotation product is same scale as displacement vector
r3d = rotation_matrix_from_vectors(T, np.insert(dp,[1],0))
r3d @ T

array([[-6.44870939e-01],
       [ 2.25363370e-16],
       [ 7.64291484e-01]])

In [684]:
# Displacement unit vector from Image 1 to Image 3, real world coordinates
V1 = r3d @ T1
V1

array([[-0.83449008],
       [ 0.17053015],
       [ 0.52397115]])

In [685]:
# Displacement unit vector from Image 2 to Image 3, real world coordinates
V2 = r3d @ (R @ T2)
V2

array([[-0.78825975],
       [ 0.6072432 ],
       [-0.09951014]])

In [686]:
# Now that unit vectors in real world are found, proceed to solve equation
unit_vectors = np.append(V1[[0,2]],-V2[[0,2]], axis=1)
# Solve this matrix and get b: V[b,c]' = D
const = np.linalg.solve(unit_vectors,dp.T)

## Checking correctness of result, by trying to add up the vector from both images and ensure they produce the same result

In [687]:
pt1 - const[0,0] * V1[[0,2]].flatten()

array([[ 51.01457279, -56.76539929]])

In [688]:
pt2 - const[1,0] * V2[[0,2]].flatten()

array([[ 51.01457279, -56.76539929]])